In [7]:
import pythoncyc
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
from bioservices import UniProt
import pandas
from io import StringIO

In [9]:
meta = pythoncyc.select_organism('meta')
#pythoncyc.sendQueryToPTools("(select-organism :org-id 'META)")
#pwydb = pandas.read_csv('../dat/biolog_metacyc.csv')

PythonCycError: Pathway Tools was unable to verify if organism (orgid) META is known in your running Pathway Tools. More specifically: Failed to create a connection to a running Pathway Tools at localhost on port 5008. Make sure Pathway Tools is running with option -python. Error code: 111, error message: Connection refused

In [4]:
meta["|PWY-5515|"]["reaction_list"]

[u'|L-XYLULOSE-REDUCTASE-RXN|',
 u'|L-ARABINITOL-4-DEHYDROGENASE-RXN|',
 u'|RXN-8772|']

In [7]:
pwydf = pandas.DataFrame(columns=["ID", "CSource", "Metacyc"])
for index,row in pwydb.iterrows():
    if pandas.isnull(row["metacyc"]):
        continue
    allmetacyc = row["metacyc"]
    csource = row["csource"]
    for i,pwy in enumerate(allmetacyc.split(";")):
        pwyid = csource+str(i)
        row = pandas.DataFrame([[pwyid,csource,pwy]], columns=["ID", "CSource", "Metacyc"])
        pwydf = pwydf.append(row)

# get EC numbers from metacyc db
ec_col = [] # ec numbers
ecu_col = [] # number of unique ec numbers (in case of multiple ec numbers for one reactions)
rea_col = [] # reaction ids
idx = -1
for index,row in pwydf.iterrows(): # index not working ???? => idx workaround
    idx += 1
    ec_col.append("")
    rea_col.append("")
    pwy = "|"+row["Metacyc"]+"|"
    if meta[pwy] == None:
        print(pwy, "Pathway does not exist")
        continue
    rea_list = meta[pwy]["reaction_list"]
    ec_list  = []
    reaid_list = []
    for r in rea_list:
        ec  = meta[r]["ec_number"]
        reaid = r
        if ec == None:
            continue
        ec_nr=len(ec)
        ec = str(",".join(ec)).replace("|","").replace("EC-","")
        ec_list.append(ec)
        reaid = reaid.replace("|","")
        reaid_list.append(",".join([reaid for i in range(0,ec_nr)]))
    ec_col[idx] = (",".join(ec_list))
    rea_col[idx]= (",".join(reaid_list))
#print(ec_list)
pwydf["EC"] = ec_col
pwydf["Metacyc_rea"] = rea_col
pwydf.to_csv("../dat/pwydf.csv", sep="\t", index=False)

('|PWY-7789|', 'Pathway does not exist')
('|PWY-7788|', 'Pathway does not exist')


In [12]:
pwydf.head()
pwydf[pwydf.CSource=="D-Galactose"]

,ID,CSource,Metacyc,EC,Metacyc_rea
0,D-Galactose0,D-Galactose,PWY-6317,"5.1.3.3,5.4.2.6,5.1.3.2,2.7.7.12,2.7.1.6","ALDOSE1EPIM-RXN,BETA-PHOSPHOGLUCOMUTASE-RXN,UD..."
0,D-Galactose1,D-Galactose,PWY-6693,"2.7.1.4,1.1.1.14,1.1.1,1.1.1,1.1.1.21","FRUCTOKINASE-RXN,RXN-7644,RXN-12080,RXN-12079,..."
0,D-Galactose2,D-Galactose,GALDEG-PWY,"3.1.1.25,1.1.1.48","GALACTONOLACTONASE-RXN,GALACTODEHYDROG-RXN"
0,D-Galactose3,D-Galactose,PWY66-422,"5.4.2.5,5.4.2.2,5.1.3.3,5.1.3.2,2.7.7.12,2.7.1.6","RXN66-532,RXN66-532,ALDOSE1EPIM-RXN,UDPGLUCEPI..."
0,D-Galactose4,D-Galactose,PWY-3821,"5.1.3.2,2.7.7.64,2.7.7.10,2.7.1.6","UDPGLUCEPIM-RXN,UTPHEXPURIDYLYLTRANS-RXN,UTPHE..."


In [13]:
u = UniProt()
for index,row in pwydf.iterrows():
    if row["EC"]== "" or row["EC"] == None:
        continue
    ec_list = row["EC"].split(",")
    for ec in ec_list:
        results = u.search(ec+"+and+reviewed:yes", columns="id,entry name, protein names, sequence", limit=10) # uniprot swissprot db (reviewed)
        if len(results) == 0:
            results = u.search(ec, columns="id,entry name, protein names, sequence", limit=10) # uniprot trembl db (unreviewed)
        if len(results) == 0:
            print("No entry found for:", row["CSource"],row["Metacyc"], ec)
            continue
        df = pandas.read_csv(StringIO(results), sep="\t")
        records = []
        for index, row2 in df.iterrows():
            record = SeqRecord(Seq(row2['Sequence'], IUPAC.protein), id=row2['Entry'], description=row2['Protein names'])
            records.append(record)
        SeqIO.write(records, "../dat/seq/"+ec+".fasta", "fasta")

('No entry found for:', 'L-Arabinose', 'PWY-7295', '4.1.2.18')
('No entry found for:', 'D-Galactose', 'PWY66-422', '5.4.2.5')
('No entry found for:', 'L-Rhamnose', 'PWY-6714', '1.1.1.378')
('No entry found for:', 'L-Rhamnose', 'PWY-6713', '1.1.1.378')
('No entry found for:', 'Sucrose', 'PWY-3801', '5.4.2.5')
('No entry found for:', 'Sucrose', 'PWY-5384', '5.4.2.5')
('No entry found for:', 'D-Trehalose', 'PWY-2723', '5.4.2.5')


In [141]:
meta["|MALTOSECAT-PWY|"]["reaction_list"]
print(meta["|BETA-PHOSPHOGLUCOMUTASE-RXN|"]["ec_number"])
print(meta["|MALTOSE-PHOSPHORYLASE-RXN|"]["ec_number"])

[u'|EC-5.4.2.6|']
[u'|EC-2.4.1.8|']


In [142]:
u = UniProt()
results = u.search("5.4.2.6+and+reviewed:yes", columns="id,entry name, protein names, sequence", limit=5)
df = pandas.read_csv(StringIO(results), sep="\t")
df

,Entry,Entry name,Protein names,Sequence
0,P71447,PGMB_LACLA,Beta-phosphoglucomutase (Beta-PGM) (EC 5.4.2.6),MFKAVLFDLDGVITDTAEYHFRAWKALAEEIGINGVDRQFNEQLKG...
1,P77366,PGMB_ECOLI,Beta-phosphoglucomutase (Beta-PGM) (EC 5.4.2.6),MKLQGVIFDLDGVITDTAHLHFQAWQQIAAEIGISIDAQFNESLKG...
2,O06995,PGMB_BACSU,Beta-phosphoglucomutase (Beta-PGM) (EC 5.4.2.6),MKAVIFDLDGVITDTAEYHFLAWKHIAEQIDIPFDRDMNERLKGIS...
